In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b1/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 4000
start_epoch = 0
train_batch = 100
test_batch = 200
lr = 0.01
schedule = [500, 1000, 2000, 3000]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b1/to_pggan/l2sp' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
iter_time = []

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot_style1')
source_train_dir = os.path.join(target_dir, '100_shot_style1_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes)
student_model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b1/checkpoint.pth.tar'


In [9]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [12]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += torch.norm(param, p=1)
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += torch.norm(param - teacher_model_weights[name], p=1)
    return sp_loss

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
#     source_set = []
#     for inputs, targets in source_train_loader:
#         source_set.append((inputs, targets))
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
#         inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        
        # cutmix
#         source_inputs, source_targets = source_set[batch_idx]
#         source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()

#         prob_delta = cm_prob_init - cm_prob_low
#         prob_step = epoch / (epochs+1) * prob_delta
#         lam = cm_prob_init - prob_step

#         rand_index = torch.randperm(inputs.size()[0]).cuda()
#         st = source_targets[rand_index]
#         tt = targets[rand_index]
#         rand_index = rand_index[st == tt]
        
#         bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
#         inputs[rand_index, :, bbx1:bbx2, bby1:bby2] = source_inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
#         lam = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
#         inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [19]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        teacher_model.load_state_dict(student_model.state_dict())
        teacher_model_weights = {}
        for name, param in teacher_model.named_parameters():
            teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 4000] LR: 0.010000
1/4 Data:0.497 | Batch:2.346 | Total:0:00:02 | ETA:0:00:08 | Loss:2.80538010597229 | top1:72.0
2/4 Data:0.001 | Batch:0.299 | Total:0:00:02 | ETA:0:00:03 | Loss:2.8741729259490967 | top1:70.5
3/4 Data:0.009 | Batch:0.270 | Total:0:00:02 | ETA:0:00:01 | Loss:2.587333838144938 | top1:72.0
4/4 Data:0.009 | Batch:0.276 | Total:0:00:03 | ETA:0:00:00 | Loss:2.3103381991386414 | top1:71.0
161/161 Data:0.000 | Batch:0.079 | Total:0:00:22 | ETA:0:00:00 | Loss:1.8900495711888108 | top1:35.345794677734375
39/39 Data:0.000 | Batch:0.131 | Total:0:00:08 | ETA:0:00:00 | Loss:1.383421995700934 | top1:63.769229888916016

Epoch: [2 | 4000] LR: 0.010600
1/4 Data:0.458 | Batch:0.811 | Total:0:00:00 | ETA:0:00:03 | Loss:1.461549997329712 | top1:54.0
2/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:02 | Loss:1.5997318029403687 | top1:51.0
3/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:01 | Loss:1.4051048358281453 | top1:53.66666793823242
4/4 Data:0.009 | Batc

4/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:00 | Loss:62.13328266143799 | top1:79.5

Epoch: [20 | 4000] LR: 0.021400
1/4 Data:0.503 | Batch:0.786 | Total:0:00:00 | ETA:0:00:03 | Loss:59.571651458740234 | top1:80.0
2/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:02 | Loss:58.91684150695801 | top1:79.0
3/4 Data:0.009 | Batch:0.271 | Total:0:00:01 | ETA:0:00:01 | Loss:58.21457163492838 | top1:79.33333587646484
4/4 Data:0.009 | Batch:0.271 | Total:0:00:01 | ETA:0:00:00 | Loss:57.49188232421875 | top1:79.5

Epoch: [21 | 4000] LR: 0.022000
1/4 Data:0.518 | Batch:0.780 | Total:0:00:00 | ETA:0:00:03 | Loss:53.742225646972656 | top1:78.0
2/4 Data:0.009 | Batch:0.259 | Total:0:00:01 | ETA:0:00:02 | Loss:52.9004020690918 | top1:78.5
3/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:01 | Loss:52.06193542480469 | top1:77.33333587646484
4/4 Data:0.009 | Batch:0.268 | Total:0:00:01 | ETA:0:00:00 | Loss:51.20620632171631 | top1:78.5

Epoch: [22 | 4000] LR: 0.022600
1/4 Data:0

1/4 Data:0.493 | Batch:0.773 | Total:0:00:00 | ETA:0:00:03 | Loss:0.7860239744186401 | top1:93.0
2/4 Data:0.009 | Batch:0.271 | Total:0:00:01 | ETA:0:00:02 | Loss:0.8130618035793304 | top1:92.5
3/4 Data:0.009 | Batch:0.271 | Total:0:00:01 | ETA:0:00:01 | Loss:0.851737399895986 | top1:88.33333587646484
4/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:00 | Loss:0.8630917370319366 | top1:87.5

Epoch: [40 | 4000] LR: 0.033400
1/4 Data:0.505 | Batch:0.781 | Total:0:00:00 | ETA:0:00:03 | Loss:0.9975807666778564 | top1:83.0
2/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:02 | Loss:0.9379417896270752 | top1:85.5
3/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:01 | Loss:0.9317355553309122 | top1:86.0
4/4 Data:0.009 | Batch:0.267 | Total:0:00:01 | ETA:0:00:00 | Loss:0.9373821467161179 | top1:86.75

Epoch: [41 | 4000] LR: 0.034000
1/4 Data:0.454 | Batch:0.753 | Total:0:00:00 | ETA:0:00:03 | Loss:0.9672231078147888 | top1:86.0
2/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | E

4/4 Data:0.009 | Batch:0.271 | Total:0:00:01 | ETA:0:00:00 | Loss:6878.6788330078125 | top1:89.5

Epoch: [58 | 4000] LR: 0.040000
1/4 Data:0.476 | Batch:0.761 | Total:0:00:00 | ETA:0:00:03 | Loss:7517.64453125 | top1:86.0
2/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:02 | Loss:7568.41064453125 | top1:86.5
3/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:01 | Loss:7596.167643229167 | top1:85.33333587646484
4/4 Data:0.009 | Batch:0.273 | Total:0:00:01 | ETA:0:00:00 | Loss:7602.368896484375 | top1:86.0

Epoch: [59 | 4000] LR: 0.040000
1/4 Data:0.497 | Batch:0.772 | Total:0:00:00 | ETA:0:00:03 | Loss:7533.8349609375 | top1:85.0
2/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:02 | Loss:7465.316650390625 | top1:86.0
3/4 Data:0.009 | Batch:0.271 | Total:0:00:01 | ETA:0:00:01 | Loss:7382.455078125 | top1:87.0
4/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:00 | Loss:7286.88525390625 | top1:86.75

Epoch: [60 | 4000] LR: 0.040000
1/4 Data:0.476 | Batch:0.765 | To

1/4 Data:0.506 | Batch:0.782 | Total:0:00:00 | ETA:0:00:03 | Loss:434.3966064453125 | top1:85.0
2/4 Data:0.009 | Batch:0.269 | Total:0:00:01 | ETA:0:00:02 | Loss:446.6905059814453 | top1:82.0
3/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:01 | Loss:457.8407897949219 | top1:83.33333587646484
4/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:00 | Loss:468.2521743774414 | top1:83.0

Epoch: [78 | 4000] LR: 0.039996
1/4 Data:0.521 | Batch:0.804 | Total:0:00:00 | ETA:0:00:03 | Loss:515.996826171875 | top1:89.0
2/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:02 | Loss:522.6015014648438 | top1:88.0
3/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:01 | Loss:527.9958089192709 | top1:87.66667175292969
4/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:00 | Loss:532.1415405273438 | top1:86.5

Epoch: [79 | 4000] LR: 0.039996
1/4 Data:0.455 | Batch:0.756 | Total:0:00:00 | ETA:0:00:03 | Loss:546.5765380859375 | top1:84.0
2/4 Data:0.009 | Batch:0.269 | Total:0:00:01 

...

3/4 Data:0.009 | Batch:0.259 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6049448450406393 | top1:95.66667175292969
4/4 Data:0.009 | Batch:0.267 | Total:0:00:01 | ETA:0:00:00 | Loss:0.5909642577171326 | top1:96.5

Epoch: [3923 | 4000] LR: 0.000000
1/4 Data:0.515 | Batch:0.783 | Total:0:00:00 | ETA:0:00:03 | Loss:0.5612933039665222 | top1:98.0
2/4 Data:0.009 | Batch:0.256 | Total:0:00:01 | ETA:0:00:02 | Loss:0.5542666912078857 | top1:98.5
3/4 Data:0.009 | Batch:0.265 | Total:0:00:01 | ETA:0:00:01 | Loss:0.574608564376831 | top1:97.33333587646484
4/4 Data:0.009 | Batch:0.269 | Total:0:00:01 | ETA:0:00:00 | Loss:0.5802172720432281 | top1:97.0

Epoch: [3924 | 4000] LR: 0.000000
1/4 Data:0.495 | Batch:0.773 | Total:0:00:00 | ETA:0:00:03 | Loss:0.7091939449310303 | top1:94.0
2/4 Data:0.009 | Batch:0.260 | Total:0:00:01 | ETA:0:00:02 | Loss:0.6425765752792358 | top1:95.5
3/4 Data:0.009 | Batch:0.257 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6070718765258789 | top1:96.66667175292969
4/4 Data:0.009 | Ba

3/4 Data:0.009 | Batch:0.249 | Total:0:00:01 | ETA:0:00:01 | Loss:0.5869519909222921 | top1:95.33333587646484
4/4 Data:0.009 | Batch:0.248 | Total:0:00:01 | ETA:0:00:00 | Loss:0.5915945917367935 | top1:95.5

Epoch: [3942 | 4000] LR: 0.000000
1/4 Data:0.504 | Batch:0.772 | Total:0:00:00 | ETA:0:00:03 | Loss:0.6107741594314575 | top1:96.0
2/4 Data:0.009 | Batch:0.248 | Total:0:00:01 | ETA:0:00:02 | Loss:0.6045401394367218 | top1:95.5
3/4 Data:0.009 | Batch:0.247 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6058229406674703 | top1:96.0
4/4 Data:0.009 | Batch:0.247 | Total:0:00:01 | ETA:0:00:00 | Loss:0.5956699252128601 | top1:96.5

Epoch: [3943 | 4000] LR: 0.000000
1/4 Data:0.534 | Batch:0.798 | Total:0:00:00 | ETA:0:00:03 | Loss:0.5773634910583496 | top1:97.0
2/4 Data:0.009 | Batch:0.254 | Total:0:00:01 | ETA:0:00:02 | Loss:0.5800564289093018 | top1:97.0
3/4 Data:0.009 | Batch:0.255 | Total:0:00:01 | ETA:0:00:01 | Loss:0.5729561249415079 | top1:97.33333587646484
4/4 Data:0.009 | Batch:0.254 | 

1/4 Data:0.509 | Batch:0.773 | Total:0:00:00 | ETA:0:00:03 | Loss:0.5935713648796082 | top1:97.0
2/4 Data:0.009 | Batch:0.272 | Total:0:00:01 | ETA:0:00:02 | Loss:0.5700569450855255 | top1:98.0
3/4 Data:0.009 | Batch:0.255 | Total:0:00:01 | ETA:0:00:01 | Loss:0.5870145559310913 | top1:97.33333587646484
4/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:00 | Loss:0.5867504626512527 | top1:97.0

Epoch: [3961 | 4000] LR: 0.000000
1/4 Data:0.518 | Batch:0.792 | Total:0:00:00 | ETA:0:00:03 | Loss:0.5981506109237671 | top1:96.0
2/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:02 | Loss:0.5714013576507568 | top1:97.5
3/4 Data:0.009 | Batch:0.252 | Total:0:00:01 | ETA:0:00:01 | Loss:0.5878100991249084 | top1:96.33333587646484
4/4 Data:0.009 | Batch:0.258 | Total:0:00:01 | ETA:0:00:00 | Loss:0.5763458460569382 | top1:97.0

Epoch: [3962 | 4000] LR: 0.000000
1/4 Data:0.505 | Batch:0.799 | Total:0:00:00 | ETA:0:00:03 | Loss:0.5588985085487366 | top1:99.0
2/4 Data:0.009 | Batch:0.266 | 

2/4 Data:0.009 | Batch:0.266 | Total:0:00:01 | ETA:0:00:02 | Loss:0.5909899771213531 | top1:97.0
3/4 Data:0.009 | Batch:0.255 | Total:0:00:01 | ETA:0:00:01 | Loss:0.5840820074081421 | top1:97.33333587646484
4/4 Data:0.009 | Batch:0.260 | Total:0:00:01 | ETA:0:00:00 | Loss:0.5767686069011688 | top1:97.5

Epoch: [3980 | 4000] LR: 0.000000
1/4 Data:0.487 | Batch:0.789 | Total:0:00:00 | ETA:0:00:03 | Loss:0.5636999011039734 | top1:99.0
2/4 Data:0.009 | Batch:0.252 | Total:0:00:01 | ETA:0:00:02 | Loss:0.5780750513076782 | top1:97.5
3/4 Data:0.009 | Batch:0.256 | Total:0:00:01 | ETA:0:00:01 | Loss:0.5754959384600321 | top1:97.66667175292969
4/4 Data:0.009 | Batch:0.255 | Total:0:00:01 | ETA:0:00:00 | Loss:0.5755888372659683 | top1:97.5

Epoch: [3981 | 4000] LR: 0.000000
1/4 Data:0.487 | Batch:0.767 | Total:0:00:00 | ETA:0:00:03 | Loss:0.5694646835327148 | top1:98.0
2/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:02 | Loss:0.6025080382823944 | top1:96.0
3/4 Data:0.009 | Batch:0.251 | 

2/4 Data:0.009 | Batch:0.263 | Total:0:00:01 | ETA:0:00:02 | Loss:0.6027008295059204 | top1:95.5
3/4 Data:0.009 | Batch:0.255 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6073632836341858 | top1:95.66667175292969
4/4 Data:0.011 | Batch:0.254 | Total:0:00:01 | ETA:0:00:00 | Loss:0.6081474423408508 | top1:95.75

Epoch: [3999 | 4000] LR: 0.000000
1/4 Data:0.505 | Batch:0.796 | Total:0:00:00 | ETA:0:00:03 | Loss:0.6099123954772949 | top1:96.0
2/4 Data:0.009 | Batch:0.270 | Total:0:00:01 | ETA:0:00:02 | Loss:0.6113116145133972 | top1:96.5
3/4 Data:0.009 | Batch:0.269 | Total:0:00:01 | ETA:0:00:01 | Loss:0.6020943919817606 | top1:97.0
4/4 Data:0.009 | Batch:0.264 | Total:0:00:01 | ETA:0:00:00 | Loss:0.608070507645607 | top1:96.75

Epoch: [4000 | 4000] LR: 0.000000
1/4 Data:0.504 | Batch:0.769 | Total:0:00:00 | ETA:0:00:03 | Loss:0.6806718707084656 | top1:93.0
2/4 Data:0.009 | Batch:0.250 | Total:0:00:01 | ETA:0:00:02 | Loss:0.6197659373283386 | top1:96.0
3/4 Data:0.009 | Batch:0.247 | Total:0:00:0